In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) #makes the display bars longer and spread out 100% across the screen width

In [2]:
!whoami
!date

nicoly
Wed Oct 14 00:36:49 PDT 2020


In [3]:
from vivarium import Artifact
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from db_queries import get_ids, get_outputs
import scipy.stats

In [4]:
#find file paths
%cd /share/costeffectiveness/results/vivarium_gates_bep/summer_run1/india/2020_09_04_17_40_19/count_data
!ls -halt

/ihme/costeffectiveness/results/vivarium_gates_bep/summer_run1/india/2020_09_04_17_40_19/count_data
total 64M
-rw-rw---- 1 kjells Domain Users 355K Sep  7 08:07 gestational_age.csv
drwxrwx--- 2 kjells Domain Users  11K Sep  7 08:07 .
-rw-rw---- 1 kjells Domain Users 1.3M Sep  7 08:07 gestational_age.hdf
-rw-rw---- 1 kjells Domain Users 344K Sep  7 08:07 birth_weight.csv
-rw-rw---- 1 kjells Domain Users 1.3M Sep  7 08:07 birth_weight.hdf
-rw-rw---- 1 kjells Domain Users 725K Sep  7 08:07 cgf_categories.csv
-rw-rw---- 1 kjells Domain Users 1.8M Sep  7 08:07 cgf_categories.hdf
-rw-rw---- 1 kjells Domain Users 390K Sep  7 08:07 cgf_z_scores.csv
-rw-rw---- 1 kjells Domain Users 1.4M Sep  7 08:07 cgf_z_scores.hdf
-rw-rw---- 1 kjells Domain Users 7.9M Sep  7 08:07 transition_count.csv
-rw-rw---- 1 kjells Domain Users 9.1M Sep  7 08:07 transition_count.hdf
-rw-rw---- 1 kjells Domain Users 7.0M Sep  7 08:07 state_person_time.csv
-rw-rw---- 1 kjells Domain Users 7.9M Sep  7 08:07 state_person_ti

In [5]:
#scenarios in each of the runs

#run1 (baseline, mmn, bep_hd_universal_i & targeted_i)
#run2 (bep_ce_universal & targeted)
#run3 (bep_ce_universal & targeted)
#run4 (baseline, mmn, bep_hd_universal_iii & targeted_iii)

#summer model
output_dirs001 = ['/share/costeffectiveness/results/vivarium_gates_bep/summer_run1/india/2020_09_04_17_40_19/count_data/',
                  '/share/costeffectiveness/results/vivarium_gates_bep/summer_run1/pakistan/2020_09_05_11_51_49/count_data/',
                  '/share/costeffectiveness/results/vivarium_gates_bep/summer_run1/mali/2020_09_04_17_51_31/count_data/',
                  '/share/costeffectiveness/results/vivarium_gates_bep/summer_run1/tanzania/2020_09_05_11_55_58/count_data/']

output_dirs002 = ['/share/costeffectiveness/results/vivarium_gates_bep/summer_run2/india/2020_09_05_12_05_16/count_data/',
                  '/share/costeffectiveness/results/vivarium_gates_bep/summer_run2/pakistan/2020_09_07_08_21_34/count_data/',
                  '/share/costeffectiveness/results/vivarium_gates_bep/summer_run2/mali/2020_09_05_12_09_07/count_data/',
                  '/share/costeffectiveness/results/vivarium_gates_bep/summer_run2/tanzania/2020_09_07_08_23_52/count_data/']


output_dirs004 = ['/share/costeffectiveness/results/vivarium_gates_bep/summer_run4/india/2020_09_07_08_39_06/count_data/',
                  '/share/costeffectiveness/results/vivarium_gates_bep/summer_run4/pakistan/2020_09_07_08_42_41/count_data/',
                  '/share/costeffectiveness/results/vivarium_gates_bep/summer_run4/mali/2020_09_07_08_41_07/count_data/',
                  '/share/costeffectiveness/results/vivarium_gates_bep/summer_run4/tanzania/2020_09_07_08_44_13/count_data/']
#location labels
locations = ['India','Pakistan','Mali','Tanzania']

#use in sort_values to make custum sort list. I sorted by region. 
sorter = ['India','mali','Mali','Tanzania']
sorterIndex = dict(zip(sorter,range(len(sorter))))
#run 001 and 002.1 

#feb model
output_dirs = ['/share/costeffectiveness/results/vivarium_gates_bep/malnutrition/india/2020_02_05_03_02_13/count_space/',
              '/share/costeffectiveness/results/vivarium_gates_bep/malnutrition/mali/2020_02_05_03_02_02/count_space/',
              '/share/costeffectiveness/results/vivarium_gates_bep/malnutrition/tanzania/2020_02_05_03_07_37/count_space/']



#location labels
locations = ['India','Pakistan','Mali','Tanzania']

#use in sort_values to make custum sort list. I sorted by region. 
sorter = ['India','Pakistan','Mali','Tanzania']
sorterIndex = dict(zip(sorter,range(len(sorter))))


## testing

In [6]:
cgf_india = pd.read_hdf('/share/costeffectiveness/results/vivarium_gates_bep/summer_run1/india/2020_09_04_17_40_19/count_data/cgf_z_scores.hdf').reset_index()
cgf_india.head()

,index,risk,treatment_group,mother_status,measure,input_draw,scenario,value
0,0,stunting,bep,malnourished,mean,3,baseline,0.000000
1,1,stunting,bep,malnourished,mean,3,bep_hd_scale_up,9.474816
2,2,stunting,bep,malnourished,mean,3,bep_hd_targeted_scale_up,9.474816
3,3,stunting,bep,malnourished,mean,3,mmn_scale_up,0.000000
4,4,stunting,bep,malnourished,mean,21,baseline,0.000000


In [7]:
deaths_india = pd.read_hdf('/share/costeffectiveness/results/vivarium_gates_bep/summer_run1/india/2020_09_04_17_40_19/count_data/deaths.hdf')
deaths_india.head()

,age_group,cause,treatment_group,mother_status,measure,input_draw,scenario,value
0,1_to_4,diarrheal_diseases,bep,malnourished,death,3,baseline,0.0
1,1_to_4,diarrheal_diseases,bep,malnourished,death,3,bep_hd_scale_up,3.0
2,1_to_4,diarrheal_diseases,bep,malnourished,death,3,bep_hd_targeted_scale_up,3.0
3,1_to_4,diarrheal_diseases,bep,malnourished,death,3,mmn_scale_up,0.0
4,1_to_4,diarrheal_diseases,bep,malnourished,death,21,baseline,0.0


In [8]:
deaths_india['cause'].value_counts()

other_causes                    8000
lower_respiratory_infections    8000
measles                         8000
diarrheal_diseases              8000
protein_energy_malnutrition     8000
Name: cause, dtype: int64

In [9]:
died_india = deaths_india.groupby(['input_draw', 'scenario','mother_status', 'treatment_group']).sum().reset_index()
died_india = died_india.rename(columns={'value':'deaths'})
died_india.head()


,input_draw,scenario,mother_status,treatment_group,deaths
0,3,baseline,malnourished,bep,0.0
1,3,baseline,malnourished,ifa,259.0
2,3,baseline,malnourished,mmn,0.0
3,3,baseline,malnourished,none,500.0
4,3,baseline,normal,bep,0.0


# CGF z-score shift check

## RUN 1 BASELINE

In [10]:
#RUN 1 (baseline, mmn, bep_hd_scale_up and bep_hd_targeted)
master_cgf_001 = pd.DataFrame()

for i in list(range(len(output_dirs001))):
    #cgf
    cgf_001 = pd.read_hdf(output_dirs001[i] + 'cgf_z_scores.hdf').reset_index().sort_values(by=['risk','measure','scenario','treatment_group','mother_status'])
    cgf_001 = cgf_001.rename(columns={'value':'zscore'})
    cgf_001['z_score'] = cgf_001['zscore'] - 10
    
    #population
    pop = pd.read_hdf(output_dirs001[i] + 'population.hdf').reset_index()
    pop = pop.where(pop['measure'] == 'total_population').dropna().rename(columns={'value':'start_population'}).drop(columns=['index','measure'])
    
    died = pd.read_hdf(output_dirs001[i] + 'deaths.hdf')
    died = died.groupby(['input_draw', 'scenario','mother_status', 'treatment_group']).sum().rename(columns={'value':'deaths'}).reset_index()
    
    pop = pop.merge(died, right_on=['scenario','treatment_group','mother_status','input_draw'], left_on=['scenario','treatment_group','mother_status','input_draw'])
    
    data = cgf_001.merge(pop, right_on=['scenario','treatment_group','mother_status','input_draw'], left_on=['scenario','treatment_group','mother_status','input_draw'])
    data = data.groupby(['risk','measure','scenario','treatment_group','mother_status']).mean().drop(columns=['input_draw','index']).reset_index() #average over input draws
    data['alive'] = data['start_population'] - data['deaths']
    data['alive_x_cgf'] = data['z_score'] * data['alive'] #cannot use this as weight because the population is the population supplemented (or at birth) whereas the z-score is among alive population at 2 years
    data['location'] = locations[i]
    data['scenario'] = data['scenario'].replace('bep_hd_scale_up', 'bep_hd_scale_up_i').replace('bep_hd_targeted_scale_up','bep_hd_targeted_scale_up_i')    
    master_cgf_001 = master_cgf_001.append(data)
    
master_cgf_001.head(10)

,risk,measure,scenario,treatment_group,mother_status,zscore,z_score,start_population,deaths,alive,alive_x_cgf,location
0,stunting,mean,baseline,bep,malnourished,0.000000,-10.000000,0.00,0.00,0.00,-0.000000,India
1,stunting,mean,baseline,bep,normal,0.000000,-10.000000,0.00,0.00,0.00,-0.000000,India
2,stunting,mean,baseline,ifa,malnourished,9.067246,-0.932754,5770.46,250.62,5519.84,-5148.654779,India
3,stunting,mean,baseline,ifa,normal,9.857427,-0.142573,28262.46,939.90,27322.56,-3895.451723,India
4,stunting,mean,baseline,mmn,malnourished,0.000000,-10.000000,0.00,0.00,0.00,-0.000000,India
5,stunting,mean,baseline,mmn,normal,0.000000,-10.000000,0.00,0.00,0.00,-0.000000,India
6,stunting,mean,baseline,none,malnourished,9.065966,-0.934034,11166.04,528.62,10637.42,-9935.707549,India
7,stunting,mean,baseline,none,normal,9.856911,-0.143089,54801.04,1888.72,52912.32,-7571.182409,India
8,stunting,mean,bep_hd_scale_up_i,bep,malnourished,9.364885,-0.635115,13444.32,486.36,12957.96,-8229.794822,India
9,stunting,mean,bep_hd_scale_up_i,bep,normal,10.154668,0.154668,65971.26,1962.38,64008.88,9900.151465,India


In [11]:
#STUNTING

stunting001 = master_cgf_001[master_cgf_001['scenario'].isin(['baseline']) & master_cgf_001['treatment_group'].isin(['ifa','none']) & master_cgf_001['location'].isin(['India'])  & master_cgf_001['risk'].isin(['stunting']) & master_cgf_001['measure'].isin(['mean'])]
stunting001 

,risk,measure,scenario,treatment_group,mother_status,zscore,z_score,start_population,deaths,alive,alive_x_cgf,location
2,stunting,mean,baseline,ifa,malnourished,9.067246,-0.932754,5770.46,250.62,5519.84,-5148.654779,India
3,stunting,mean,baseline,ifa,normal,9.857427,-0.142573,28262.46,939.90,27322.56,-3895.451723,India
6,stunting,mean,baseline,none,malnourished,9.065966,-0.934034,11166.04,528.62,10637.42,-9935.707549,India
7,stunting,mean,baseline,none,normal,9.856911,-0.143089,54801.04,1888.72,52912.32,-7571.182409,India


In [12]:
#WASTING

wasting001 = master_cgf_001[master_cgf_001['scenario'].isin(['baseline']) & master_cgf_001['treatment_group'].isin(['ifa','none']) & master_cgf_001['location'].isin(['India'])  & master_cgf_001['risk'].isin(['wasting']) & master_cgf_001['measure'].isin(['mean'])]
wasting001 

,risk,measure,scenario,treatment_group,mother_status,zscore,z_score,start_population,deaths,alive,alive_x_cgf,location
66,wasting,mean,baseline,ifa,malnourished,9.067246,-0.932754,5770.46,250.62,5519.84,-5148.654779,India
67,wasting,mean,baseline,ifa,normal,9.857427,-0.142573,28262.46,939.90,27322.56,-3895.451723,India
70,wasting,mean,baseline,none,malnourished,9.065966,-0.934034,11166.04,528.62,10637.42,-9935.707549,India
71,wasting,mean,baseline,none,normal,9.856911,-0.143089,54801.04,1888.72,52912.32,-7571.182409,India


In [13]:
#STUNTING

stunting001 = master_cgf_001[master_cgf_001['scenario'].isin(['bep_hd_scale_up_i']) & master_cgf_001['treatment_group'].isin(['bep','none']) & master_cgf_001['location'].isin(['India'])  & master_cgf_001['risk'].isin(['stunting']) & master_cgf_001['measure'].isin(['mean'])]
stunting001 

,risk,measure,scenario,treatment_group,mother_status,zscore,z_score,start_population,deaths,alive,alive_x_cgf,location
8,stunting,mean,bep_hd_scale_up_i,bep,malnourished,9.364885,-0.635115,13444.32,486.36,12957.96,-8229.794822,India
9,stunting,mean,bep_hd_scale_up_i,bep,normal,10.154668,0.154668,65971.26,1962.38,64008.88,9900.151465,India
14,stunting,mean,bep_hd_scale_up_i,none,malnourished,9.065016,-0.934984,3492.18,165.40,3326.78,-3110.487122,India
15,stunting,mean,bep_hd_scale_up_i,none,normal,9.859471,-0.140529,17092.24,588.80,16503.44,-2319.215989,India


In [14]:
#WASTING

wastingg001 = master_cgf_001[master_cgf_001['scenario'].isin(['bep_hd_scale_up_i']) & master_cgf_001['treatment_group'].isin(['bep','none']) & master_cgf_001['location'].isin(['India'])  & master_cgf_001['risk'].isin(['wasting']) & master_cgf_001['measure'].isin(['mean'])]
wastingg001 

,risk,measure,scenario,treatment_group,mother_status,zscore,z_score,start_population,deaths,alive,alive_x_cgf,location
72,wasting,mean,bep_hd_scale_up_i,bep,malnourished,9.364885,-0.635115,13444.32,486.36,12957.96,-8229.794822,India
73,wasting,mean,bep_hd_scale_up_i,bep,normal,10.154668,0.154668,65971.26,1962.38,64008.88,9900.151465,India
78,wasting,mean,bep_hd_scale_up_i,none,malnourished,9.065016,-0.934984,3492.18,165.40,3326.78,-3110.487122,India
79,wasting,mean,bep_hd_scale_up_i,none,normal,9.859471,-0.140529,17092.24,588.80,16503.44,-2319.215989,India


## RUN 2 -scenarios CE universal/targeted

In [15]:
#RUN 2 (bep_ce_universal & targeted)

master_cgf_002 = pd.DataFrame()

for i in list(range(len(output_dirs002))):
    #cgf
    cgf_002 = pd.read_hdf(output_dirs002[i] + 'cgf_z_scores.hdf').reset_index().sort_values(by=['risk','measure','scenario','treatment_group','mother_status'])
    cgf_002 = cgf_002.rename(columns={'value':'zscore'})
    cgf_002['z_score'] = cgf_002['zscore'] - 10
    
    #population
    pop = pd.read_hdf(output_dirs002[i] + 'population.hdf').reset_index()
    pop = pop.where(pop['measure'] == 'total_population').dropna().rename(columns={'value':'start_population'}).drop(columns=['index','measure'])
    
    died = pd.read_hdf(output_dirs002[i] + 'deaths.hdf')
    died = died.groupby(['input_draw', 'scenario','mother_status', 'treatment_group']).sum().rename(columns={'value':'deaths'}).reset_index()
    
    pop = pop.merge(died, right_on=['scenario','treatment_group','mother_status','input_draw'], left_on=['scenario','treatment_group','mother_status','input_draw'])
    
    data = cgf_002.merge(pop, right_on=['scenario','treatment_group','mother_status','input_draw'], left_on=['scenario','treatment_group','mother_status','input_draw'])
    data = data.groupby(['risk','measure','scenario','treatment_group','mother_status']).mean().drop(columns=['input_draw','index']).reset_index() #average over input draws
    data['alive'] = data['start_population'] - data['deaths']
    data['alive_x_cgf'] = data['z_score'] * data['alive'] #cannot use this as weight because the population is the population supplemented (or at birth) whereas the z-score is among alive population at 2 years
    data['location'] = locations[i]
    master_cgf_002 = master_cgf_002.append(data)
    
master_cgf_002.head()

,risk,measure,scenario,treatment_group,mother_status,zscore,z_score,start_population,deaths,alive,alive_x_cgf,location
0,stunting,mean,bep_ce_scale_up,bep,malnourished,9.694349,-0.305651,13444.32,469.06,12975.26,-3965.896385,India
1,stunting,mean,bep_ce_scale_up,bep,normal,9.693313,-0.306687,65971.26,2135.04,63836.22,-19577.754731,India
2,stunting,mean,bep_ce_scale_up,ifa,malnourished,0.000000,-10.000000,0.00,0.00,0.00,-0.000000,India
3,stunting,mean,bep_ce_scale_up,ifa,normal,0.000000,-10.000000,0.00,0.00,0.00,-0.000000,India
4,stunting,mean,bep_ce_scale_up,mmn,malnourished,0.000000,-10.000000,0.00,0.00,0.00,-0.000000,India


In [16]:
# STUNTING

stunting002 = master_cgf_002[master_cgf_002['scenario'].isin(['bep_ce_targeted_scale_up']) & master_cgf_002['treatment_group'].isin(['bep','mmn','none']) & master_cgf_002['location'].isin(['India'])  & master_cgf_002['risk'].isin(['stunting']) & master_cgf_002['measure'].isin(['mean'])]
stunting002 

,risk,measure,scenario,treatment_group,mother_status,zscore,z_score,start_population,deaths,alive,alive_x_cgf,location
8,stunting,mean,bep_ce_targeted_scale_up,bep,malnourished,9.694349,-0.305651,13444.32,469.06,12975.26,-3965.896385,India
9,stunting,mean,bep_ce_targeted_scale_up,bep,normal,0.000000,-10.000000,0.00,0.00,0.00,-0.000000,India
12,stunting,mean,bep_ce_targeted_scale_up,mmn,malnourished,0.000000,-10.000000,0.00,0.00,0.00,-0.000000,India
13,stunting,mean,bep_ce_targeted_scale_up,mmn,normal,9.693301,-0.306699,65971.26,2190.18,63781.08,-19561.615362,India
14,stunting,mean,bep_ce_targeted_scale_up,none,malnourished,9.692166,-0.307834,3492.18,149.72,3342.46,-1028.924402,India
15,stunting,mean,bep_ce_targeted_scale_up,none,normal,9.696224,-0.303776,17092.24,601.96,16490.28,-5009.348052,India


In [17]:
# WASTING

wasting002 = master_cgf_002[master_cgf_002['scenario'].isin(['bep_ce_scale_up']) & master_cgf_002['treatment_group'].isin(['bep','mmn','none']) & master_cgf_002['location'].isin(['India'])  & master_cgf_002['risk'].isin(['wasting']) & master_cgf_002['measure'].isin(['mean'])]
wasting002

,risk,measure,scenario,treatment_group,mother_status,zscore,z_score,start_population,deaths,alive,alive_x_cgf,location
32,wasting,mean,bep_ce_scale_up,bep,malnourished,9.694349,-0.305651,13444.32,469.06,12975.26,-3965.896385,India
33,wasting,mean,bep_ce_scale_up,bep,normal,9.693313,-0.306687,65971.26,2135.04,63836.22,-19577.754731,India
36,wasting,mean,bep_ce_scale_up,mmn,malnourished,0.000000,-10.000000,0.00,0.00,0.00,-0.000000,India
37,wasting,mean,bep_ce_scale_up,mmn,normal,0.000000,-10.000000,0.00,0.00,0.00,-0.000000,India
38,wasting,mean,bep_ce_scale_up,none,malnourished,9.692166,-0.307834,3492.18,149.72,3342.46,-1028.924402,India
39,wasting,mean,bep_ce_scale_up,none,normal,9.696224,-0.303776,17092.24,601.96,16490.28,-5009.348052,India


## RUN 4 - scenarios HD universal/targeted

In [18]:
#RUN 4 (bep_ce_universal & targeted)

master_cgf_004 = pd.DataFrame()

for i in list(range(len(output_dirs004))):
    #cgf
    cgf_004 = pd.read_hdf(output_dirs004[i] + 'cgf_z_scores.hdf').reset_index().sort_values(by=['risk','measure','scenario','treatment_group','mother_status'])
    cgf_004 = cgf_004.rename(columns={'value':'zscore'})
    cgf_004['z_score'] = cgf_004['zscore'] - 10
    
    #population
    pop = pd.read_hdf(output_dirs004[i] + 'population.hdf').reset_index()
    pop = pop.where(pop['measure'] == 'total_population').dropna().rename(columns={'value':'start_population'}).drop(columns=['index','measure'])
    
    died = pd.read_hdf(output_dirs004[i] + 'deaths.hdf')
    died = died.groupby(['input_draw', 'scenario','mother_status', 'treatment_group']).sum().rename(columns={'value':'deaths'}).reset_index()
    
    pop = pop.merge(died, right_on=['scenario','treatment_group','mother_status','input_draw'], left_on=['scenario','treatment_group','mother_status','input_draw'])
    
    data = cgf_004.merge(pop, right_on=['scenario','treatment_group','mother_status','input_draw'], left_on=['scenario','treatment_group','mother_status','input_draw'])
    data = data.groupby(['risk','measure','scenario','treatment_group','mother_status']).mean().drop(columns=['input_draw','index']).reset_index() #average over input draws
    data['alive'] = data['start_population'] - data['deaths']
    data['alive_x_cgf'] = data['z_score'] * data['alive'] #cannot use this as weight because the population is the population supplemented (or at birth) whereas the z-score is among alive population at 2 years
    data['location'] = locations[i]
    data['scenario'] = data['scenario'].replace('bep_hd_scale_up', 'bep_hd_scale_up_iii').replace('bep_hd_targeted_scale_up','bep_hd_targeted_scale_up_iii')    
    master_cgf_004 = master_cgf_004.append(data)
    
master_cgf_004.head()

,risk,measure,scenario,treatment_group,mother_status,zscore,z_score,start_population,deaths,alive,alive_x_cgf,location
0,stunting,mean,bep_hd_scale_up_iii,bep,malnourished,10.008805,0.008805,13444.32,439.80,13004.52,114.506126,India
1,stunting,mean,bep_hd_scale_up_iii,bep,normal,10.007342,0.007342,65971.26,2000.14,63971.12,469.664947,India
2,stunting,mean,bep_hd_scale_up_iii,ifa,malnourished,0.000000,-10.000000,0.00,0.00,0.00,-0.000000,India
3,stunting,mean,bep_hd_scale_up_iii,ifa,normal,0.000000,-10.000000,0.00,0.00,0.00,-0.000000,India
4,stunting,mean,bep_hd_scale_up_iii,mmn,malnourished,0.000000,-10.000000,0.00,0.00,0.00,-0.000000,India


In [19]:
#WASTING

wasting004 = master_cgf_004[master_cgf_004['scenario'].isin(['bep_hd_scale_up_iii']) & master_cgf_004['treatment_group'].isin(['bep','none']) & master_cgf_004['location'].isin(['India'])  & master_cgf_004['risk'].isin(['stunting']) & master_cgf_004['measure'].isin(['mean'])]
wasting004 

,risk,measure,scenario,treatment_group,mother_status,zscore,z_score,start_population,deaths,alive,alive_x_cgf,location
0,stunting,mean,bep_hd_scale_up_iii,bep,malnourished,10.008805,0.008805,13444.32,439.80,13004.52,114.506126,India
1,stunting,mean,bep_hd_scale_up_iii,bep,normal,10.007342,0.007342,65971.26,2000.14,63971.12,469.664947,India
6,stunting,mean,bep_hd_scale_up_iii,none,malnourished,9.719043,-0.280957,3492.18,149.38,3342.80,-939.181567,India
7,stunting,mean,bep_hd_scale_up_iii,none,normal,9.714250,-0.285750,17092.24,601.70,16490.54,-4712.173295,India


# PREVALENCE 

# ================= FEB 2020 model =====================================

 I think the population needs to be updated to those alive

In [20]:
master_stunting = pd.DataFrame()
master_wasting = pd.DataFrame()

for i in list(range(len(output_dirs))):
    cgf = pd.read_hdf(output_dirs[i] + 'category_counts.hdf').reset_index()
    cgf = (cgf.where(cgf['sex'] == 'male').dropna()
           .groupby(['input_draw','cgf_risk','risk_category','malnourishment_category']).sum()
           .reset_index())
    pop = (cgf.where(cgf['cgf_risk'] == 'stunting').dropna()
          .groupby(['input_draw','malnourishment_category'], as_index=False).sum()
          .rename(columns={'category_counts':'population'}))
    cgf = cgf.where(cgf['risk_category'] == ('cat1' or 'cat2'))
    data = cgf.merge(pop, right_on=['input_draw','malnourishment_category'], 
                     left_on=['input_draw','malnourishment_category'])
    stratified = data.groupby(['input_draw','malnourishment_category','cgf_risk']).sum().reset_index()
    overall = data.groupby(['input_draw','cgf_risk']).sum().reset_index()
    overall['malnourishment_category'] = 'all'
    data = stratified.append(overall)  
    
    stunted = (data.where(data['cgf_risk'] == 'stunting').dropna())
    stunted['percent'] = stunted['category_counts'] / stunted['population']
    stunted = (stunted.groupby(['malnourishment_category']).mean()
              .drop(columns=['input_draw','category_counts','population'])
               .reset_index()) 
    stunted['location'] = locations[i]
    master_stunting = master_stunting.append(stunted)   
    
    wasted = (data.where(data['cgf_risk'] == 'wasting').dropna())
    wasted['percent'] = wasted['category_counts'] / wasted['population']
    wasted = (wasted.groupby(['malnourishment_category']).mean()
              .drop(columns=['input_draw','category_counts','population'])
               .reset_index()) 
    wasted['location'] = locations[i]
    master_wasting = master_wasting.append(wasted)
    master_wasting

/share/code/nicoly/miniconda/envs/gates_bep/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [21]:
master_wasting

,malnourishment_category,percent,location
0,all,0.055677,India
1,malnourished,0.142786,India
2,not_malnourished,0.038791,India
0,all,0.030601,Pakistan
1,malnourished,0.090116,Pakistan
2,not_malnourished,0.024041,Pakistan
0,all,0.014787,Mali
1,malnourished,0.041370,Mali
2,not_malnourished,0.012091,Mali


In [22]:
master_stunting

,malnourishment_category,percent,location
0,all,0.091158,India
1,malnourished,0.200051,India
2,not_malnourished,0.069993,India
0,all,0.038440,Pakistan
1,malnourished,0.131865,Pakistan
2,not_malnourished,0.028073,Pakistan
0,all,0.059996,Mali
1,malnourished,0.138289,Mali
2,not_malnourished,0.052009,Mali
